**Needed Libraries**


In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras_tuner
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-2-a25c4756a4d2>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


**Loading Dataset**

In [3]:
(x, y), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


**Implementing the model**

In [4]:
def build_model(hp):
    model = tf.keras.Sequential() # my model is a sequential one

    # start tuning the number of layers
    for i in range(hp.Int("cnv_number_of_layers", 1, 5)):
        model.add(
            layers.Conv2D(
                # start tuning the number of filters
                filters = hp.Int(f"filters{i}", min_value=32, max_value=256, step=32),
                kernel_size = (3,3),
                activation = 'relu',
                padding = 'same'
            )
        )
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    for i in range(hp.Int("number_of_layers", 1, 5)):
        model.add(
            layers.Dense(
                # start tuning the number of units
                units = hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
                activation = hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), # using Adam optimizer
        loss = "categorical_crossentropy",
        metrics = ["accuracy"],
    )
    return model

**Tunner Hyperparemeter**

In [5]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective = "val_accuracy",
    max_trials =3,
    executions_per_trial = 2,
    overwrite = True,
    directory = "Q1",
    project_name = "Q1",
)

**Start using tunner**

In [6]:
tuner.search(x_train, y_train, epochs = 3, validation_data=(x_val, y_val))

Trial 3 Complete [00h 27m 44s]
val_accuracy: 0.9828999936580658

Best val_accuracy So Far: 0.9875500202178955
Total elapsed time: 01h 02m 43s


**Best model**

In [7]:
# best 2 models
models = tuner.get_best_models(num_models = 2)
best_model = models[0]

# Build model
best_model.build(input_shape = x_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (50000, 28, 28, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2  (50000, 14, 14, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (50000, 14, 14, 256)      73984     
                                                                 
 max_pooling2d_1 (MaxPoolin  (50000, 7, 7, 256)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (50000, 12544)            0         
                                                                 
 dense (Dense)               (50000, 96)               1

**Training on the best model**

In [8]:
best_model.fit(x_train, y_train, epochs = 10, validation_split=0.2)

Epoch 1/10
1250/1250 [==============================] - 144s 114ms/step - loss: 0.0352 - accuracy: 0.9902 - val_loss: 0.0305 - val_accuracy: 0.9907
Epoch 2/10
1250/1250 [==============================] - 137s 110ms/step - loss: 0.0259 - accuracy: 0.9930 - val_loss: 0.0517 - val_accuracy: 0.9872
Epoch 3/10
1250/1250 [==============================] - 135s 108ms/step - loss: 0.0222 - accuracy: 0.9947 - val_loss: 0.0373 - val_accuracy: 0.9905
Epoch 4/10
1250/1250 [==============================] - 137s 110ms/step - loss: 0.0173 - accuracy: 0.9956 - val_loss: 0.0352 - val_accuracy: 0.9915
Epoch 5/10
1250/1250 [==============================] - 139s 111ms/step - loss: 0.0181 - accuracy: 0.9951 - val_loss: 0.0566 - val_accuracy: 0.9884
Epoch 6/10
1250/1250 [==============================] - 137s 110ms/step - loss: 0.0146 - accuracy: 0.9964 - val_loss: 0.0417 - val_accuracy: 0.9915
Epoch 7/10
1250/1250 [==============================] - 137s 109ms/step - loss: 0.0129 - accuracy: 0.9966 - val_

**Result**

In [9]:
# evaluation
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)

# test accuracy
print(f'Test accuracy is: {test_accuracy}')

313/313 [==============================] - 9s 27ms/step - loss: 0.0622 - accuracy: 0.9900
Test accuracy is: 0.9900000095367432
